In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import pickle

## Pandas Settings

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Import Data

In [2]:
#no2 = pd.read_csv('../Datos/Calidad_Aire/Mediciones_AMB_NO2.csv')
#o3 = pd.read_csv('../Datos/Datos_Procesados/Mediciones_AMB_O3.csv')

In [3]:
df = pd.read_csv('../Datos/Datos_Procesados/Mediciones_AMB_NO2_O3.csv')
df.sort_values(by=['FECHA'],inplace=True)

Inspección dataframe:

In [4]:
df.head()

,CODI EOI,NOM ESTACIO,FECHA,DIA,TIPO_DIA,NO2,O3
0,8015021,Badalona,2018-01-01 00:00:00,0,Laboral,16.0,41.0
76256,8019050,Barcelona (Ciutadella),2018-01-01 00:00:00,0,Laboral,14.0,47.0
25597,8019043,Barcelona (Eixample),2018-01-01 00:00:00,0,Laboral,16.0,45.0
126746,8019057,Barcelona (Palau Reial),2018-01-01 00:00:00,0,Laboral,5.0,55.0
172632,8089005,Gavà,2018-01-01 00:00:00,0,Laboral,5.0,67.0


In [5]:
carretera = pd.read_csv('../Datos/Datos_Procesados/Estacion_Buffes_LongitudCarretera.csv')
usosSuelo = pd.read_csv('../Datos/Datos_Procesados/Estacion_Buffers_AreaUsosSuelo_urbanAtlas.csv')
poblacion = pd.read_csv('../Datos/Datos_Procesados/Estacion_Buffers_Poblacion_worldpop_noAdj.csv')
meteo1819 = pd.read_csv('../Datos/Datos_Procesados/MeteoEstaciones_1819.csv')
meteo2021 = pd.read_csv('../Datos/Datos_Procesados/MeteoEstaciones_2021.csv')
meteo = pd.concat([meteo1819,meteo2021])

# Analisis NULOS

In [6]:
listNull = []
for i,v in enumerate(df['CODI EOI'].unique()):
    df_aux = df[df['CODI EOI']==v]
    listNull.append([v,df_aux.shape[0],df_aux.isna().sum()[3]])
df_auxNull = pd.DataFrame(listNull,columns=['CODI EOI','NumRows','Nulls'])
df_auxNull['%Null'] = ((df_auxNull['Nulls']/df_auxNull['NumRows'])*100).round(2)
df_auxNull

,CODI EOI,NumRows,Nulls,%Null
0,8015021,25597,0,0.0
1,8019050,25334,0,0.0
2,8019043,25092,0,0.0
3,8019057,22728,0,0.0
4,8089005,25811,0,0.0
5,8125002,24996,0,0.0
6,8194008,25371,0,0.0
7,8205002,25118,0,0.0
8,8263001,25184,0,0.0
9,8301004,25609,0,0.0


# UNIR DF

In [7]:
df.shape

(350036, 7)

Unir variables meteo:

In [8]:
df1 = pd.merge(df, meteo, how="left", left_on=['CODI EOI','FECHA',],right_on=['COD','valid_time'])
df1.drop(['Unnamed: 0', 'COD', 'latitude', 'longitude', 'valid_time'],axis=1,inplace=True)

Filtrar valores posteriores a 2021-03-01 00:00:00, dado que no hay datos de meteo posteriores:

In [9]:
df1 = df1[df1['FECHA']<='2021-03-01 00:00:00']

In [10]:
df1.shape

(337815, 12)

Unir variables población y borrar columnas superfluas:

In [11]:
df1 = pd.merge(df1, poblacion, how="left", on='CODI EOI')
df1.drop(['Unnamed: 0', 'ESTACION'],axis=1,inplace=True)

In [12]:
df1.shape

(337815, 26)

Unir variables longitud carreteras y borrar columnas superfluas:

In [13]:
df1 = pd.merge(df1, carretera, how="left", on='CODI EOI')
df1.drop(['Unnamed: 0', 'ESTACION'],axis=1,inplace=True)

Unir variables superficie uso de suelo y borrar columnas superfluas:

In [14]:
df1 = pd.merge(df1, usosSuelo, how="left", on='CODI EOI')
df1.drop(['Unnamed: 0', 'ESTACION'],axis=1,inplace=True)

## Comprobar número nulos por Estación de predictores:

In [15]:
df1.isnull().sum() > 0

CODI EOI                    False
NOM ESTACIO                 False
FECHA                       False
DIA                         False
TIPO_DIA                    False
NO2                         False
O3                          False
u10                         False
v10                         False
t2m                         False
ssrd                        False
tp                          False
b25                         False
b50                         False
b100                        False
b300                        False
b500                        False
b1000                       False
b1500                       False
b2000                       False
b2500                       False
b3000                       False
b3500                       False
b4000                       False
b4500                       False
b5000                       False
residential_b25             False
service_b25                 False
tertiary_b25                False
primary_b50   

# Nombres Columnas

In [16]:
for i,col in enumerate(df1.columns):
    print(f'{i+1} - {col}')

1 - CODI EOI
2 - NOM ESTACIO
3 - FECHA
4 - DIA
5 - TIPO_DIA
6 - NO2
7 - O3
8 - u10
9 - v10
10 - t2m
11 - ssrd
12 - tp
13 - b25
14 - b50
15 - b100
16 - b300
17 - b500
18 - b1000
19 - b1500
20 - b2000
21 - b2500
22 - b3000
23 - b3500
24 - b4000
25 - b4500
26 - b5000
27 - residential_b25
28 - service_b25
29 - tertiary_b25
30 - primary_b50
31 - residential_b50
32 - secondary_b50
33 - service_b50
34 - tertiary_b50
35 - unclassified_b50
36 - primary_b100
37 - residential_b100
38 - secondary_b100
39 - service_b100
40 - tertiary_b100
41 - unclassified_b100
42 - motorway_b300
43 - primary_b300
44 - residential_b300
45 - secondary_b300
46 - service_b300
47 - tertiary_b300
48 - trunk_b300
49 - unclassified_b300
50 - motorway_b500
51 - primary_b500
52 - residential_b500
53 - secondary_b500
54 - service_b500
55 - tertiary_b500
56 - trunk_b500
57 - unclassified_b500
58 - motorway_b1000
59 - primary_b1000
60 - residential_b1000
61 - secondary_b1000
62 - service_b1000
63 - tertiary_b1000
64 - trunk_b1

# PREPARACION DF

## Dicotomizar **TIPO_DIA**:

In [17]:
df1["TIPO_DIA"].replace({"Laboral": True, "Festivo": False}, inplace=True)
df1['TIPO_DIA'] = df1['TIPO_DIA'].astype(bool)

## Introducir Columnas Memoria:

In [18]:
df1.sort_values(by=['CODI EOI','FECHA'],inplace=True)

In [19]:
df2 = pd.DataFrame()
for i,est in enumerate(df1['CODI EOI'].unique()):
    dfaux = df1[df1['CODI EOI']==est]
    dfaux['NO2_t_1'] = dfaux['NO2'].shift(periods=1,fill_value=0)
    dfaux['NO2_diff'] = dfaux['NO2'].diff().fillna(0)
    df2 = df2.append(dfaux)
    '''if(i==0):
        df2 = dfaux.copy()
    else:
        df2.a'''

<ipython-input-19-184c7e654a62>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfaux['NO2_t_1'] = dfaux['NO2'].shift(periods=1,fill_value=0)
<ipython-input-19-184c7e654a62>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfaux['NO2_diff'] = dfaux['NO2'].diff().fillna(0)
<ipython-input-19-184c7e654a62>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

<ipython-input-19-184c7e654a62>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfaux['NO2_t_1'] = dfaux['NO2'].shift(periods=1,fill_value=0)
<ipython-input-19-184c7e654a62>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfaux['NO2_diff'] = dfaux['NO2'].diff().fillna(0)
<ipython-input-19-184c7e654a62>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

## Reordenar columnas y selección de columnas de interes:

In [20]:
df2 = df2.iloc[:,np.r_[0,1,2,6,5,435,436,4,7:435]]

In [21]:
df2['CODI EOI'].unique()

array([8015021, 8019043, 8019044, 8019050, 8019054, 8019057, 8019058,
       8089005, 8125002, 8169009, 8194008, 8205002, 8263001, 8301004],
      dtype=int64)

In [22]:
df2.shape

(337815, 436)

In [29]:
for i,col in enumerate(df2.columns):
    print(f'{i+1} - {col}')

1 - CODI EOI
2 - NOM ESTACIO
3 - FECHA
4 - O3
5 - NO2
6 - NO2_t_1
7 - NO2_diff
8 - TIPO_DIA
9 - u10
10 - v10
11 - t2m
12 - ssrd
13 - tp
14 - b25
15 - b50
16 - b100
17 - b300
18 - b500
19 - b1000
20 - b1500
21 - b2000
22 - b2500
23 - b3000
24 - b3500
25 - b4000
26 - b4500
27 - b5000
28 - residential_b25
29 - service_b25
30 - tertiary_b25
31 - primary_b50
32 - residential_b50
33 - secondary_b50
34 - service_b50
35 - tertiary_b50
36 - unclassified_b50
37 - primary_b100
38 - residential_b100
39 - secondary_b100
40 - service_b100
41 - tertiary_b100
42 - unclassified_b100
43 - motorway_b300
44 - primary_b300
45 - residential_b300
46 - secondary_b300
47 - service_b300
48 - tertiary_b300
49 - trunk_b300
50 - unclassified_b300
51 - motorway_b500
52 - primary_b500
53 - residential_b500
54 - secondary_b500
55 - service_b500
56 - tertiary_b500
57 - trunk_b500
58 - unclassified_b500
59 - motorway_b1000
60 - primary_b1000
61 - residential_b1000
62 - secondary_b1000
63 - service_b1000
64 - tertiary_b

In [26]:
df2.head()

,CODI EOI,NOM ESTACIO,FECHA,O3,NO2,NO2_t_1,NO2_diff,TIPO_DIA,u10,v10,t2m,ssrd,tp,b25,b50,b100,b300,b500,b1000,b1500,b2000,b2500,b3000,b3500,b4000,b4500,b5000,residential_b25,service_b25,tertiary_b25,primary_b50,residential_b50,secondary_b50,service_b50,tertiary_b50,unclassified_b50,primary_b100,residential_b100,secondary_b100,service_b100,tertiary_b100,unclassified_b100,motorway_b300,primary_b300,residential_b300,secondary_b300,service_b300,tertiary_b300,trunk_b300,unclassified_b300,motorway_b500,primary_b500,residential_b500,secondary_b500,service_b500,tertiary_b500,trunk_b500,unclassified_b500,motorway_b1000,primary_b1000,residential_b1000,secondary_b1000,service_b1000,tertiary_b1000,trunk_b1000,unclassified_b1000,motorway_b1500,primary_b1500,residential_b1500,secondary_b1500,service_b1500,tertiary_b1500,trunk_b1500,unclassified_b1500,motorway_b2000,primary_b2000,residential_b2000,secondary_b2000,service_b2000,tertiary_b2000,trunk_b2000,unclassified_b2000,motorway_b2500,primary_b2500,residential_b2500,secondary_b2500,service_b2500,tertiary_b2500,trunk_b2500,unclassified_b2500,motorway_b3000,primary_b3000,residential_b3000,secondary_b3000,service_b3000,tertiary_b3000,trunk_b3000,unclassified_b3000,motorway_b3500,primary_b3500,residential_b3500,secondary_b3500,service_b3500,tertiary_b3500,trunk_b3500,unclassified_b3500,motorway_b4000,primary_b4000,residential_b4000,secondary_b4000,service_b4000,tertiary_b4000,trunk_b4000,unclassified_b4000,motorway_b4500,primary_b4500,residential_b4500,secondary_b4500,service_b4500,tertiary_b4500,trunk_b4500,unclassified_b4500,motorway_b5000,primary_b5000,residential_b5000,secondary_b5000,service_b5000,tertiary_b5000,trunk_b5000,unclassified_b5000,contUrbFab_b25,discDensUrbFab_b25,discLowDensUrbFab_b25,discMedDensUrbFab_b25,discVLowDensUrbFab_b25,forest_b25,grUrbAreas_b25,herbVegetation_b25,industryCommercial_b25,otherRoads_b25,pastures_b25,railways_b25,sports_b25,annualCrops_b50,contUrbFab_b50,discDensUrbFab_b50,discLowDensUrbFab_b50,discMedDensUrbFab_b50,discVLowDensUrbFab_b50,forest_b50,grUrbAreas_b50,herbVegetation_b50,industryCommercial_b50,landNoUse_b50,otherRoads_b50,pastures_b50,railways_b50,sports_b50,water_b50,annualCrops_b100,contUrbFab_b100,discDensUrbFab_b100,discLowDensUrbFab_b100,discMedDensUrbFab_b100,discVLowDensUrbFab_b100,fastRoads_b100,forest_b100,grUrbAreas_b100,herbVegetation_b100,industryCommercial_b100,landNoUse_b100,mininDump_b100,otherRoads_b100,pastures_b100,railways_b100,sports_b100,water_b100,annualCrops_b300,constructionSites_b300,contUrbFab_b300,discDensUrbFab_b300,discLowDensUrbFab_b300,discMedDensUrbFab_b300,discVLowDensUrbFab_b300,fastRoads_b300,forest_b300,grUrbAreas_b300,herbVegetation_b300,industryCommercial_b300,isolStruct_b300,landNoUse_b300,mininDump_b300,otherRoads_b300,pastures_b300,railways_b300,sports_b300,water_b300,annualCrops_b500,constructionSites_b500,contUrbFab_b500,discDensUrbFab_b500,discLowDensUrbFab_b500,discMedDensUrbFab_b500,discVLowDensUrbFab_b500,fastRoads_b500,forest_b500,grUrbAreas_b500,herbVegetation_b500,industryCommercial_b500,isolStruct_b500,landNoUse_b500,mininDump_b500,otherRoads_b500,pastures_b500,permanentCrops_b500,railways_b500,sports_b500,water_b500,airports_b1000,annualCrops_b1000,constructionSites_b1000,contUrbFab_b1000,discDensUrbFab_b1000,discLowDensUrbFab_b1000,discMedDensUrbFab_b1000,discVLowDensUrbFab_b1000,fastRoads_b1000,forest_b1000,grUrbAreas_b1000,herbVegetation_b1000,industryCommercial_b1000,isolStruct_b1000,landNoUse_b1000,mininDump_b1000,opSpaceNoVeg_b1000,otherRoads_b1000,pastures_b1000,permanentCrops_b1000,railways_b1000,sports_b1000,water_b1000,airports_b1500,annualCrops_b1500,constructionSites_b1500,contUrbFab_b1500,discDensUrbFab_b1500,discLowDensUrbFab_b1500,discMedDensUrbFab_b1500,discVLowDensUrbFab_b1500,fastRoads_b1500,forest_b1500,grUrbAreas_b1500,herbVegetation_b1500,industryCommercial_b1500,isolStruct_b1500,landNoUse_b1500,mininDump_b1500,opSpaceNoVeg_b1500,otherRoads_b1500,pastures_b1500,permanen

In [27]:
df2.tail()

,CODI EOI,NOM ESTACIO,FECHA,O3,NO2,NO2_t_1,NO2_diff,TIPO_DIA,u10,v10,t2m,ssrd,tp,b25,b50,b100,b300,b500,b1000,b1500,b2000,b2500,b3000,b3500,b4000,b4500,b5000,residential_b25,service_b25,tertiary_b25,primary_b50,residential_b50,secondary_b50,service_b50,tertiary_b50,unclassified_b50,primary_b100,residential_b100,secondary_b100,service_b100,tertiary_b100,unclassified_b100,motorway_b300,primary_b300,residential_b300,secondary_b300,service_b300,tertiary_b300,trunk_b300,unclassified_b300,motorway_b500,primary_b500,residential_b500,secondary_b500,service_b500,tertiary_b500,trunk_b500,unclassified_b500,motorway_b1000,primary_b1000,residential_b1000,secondary_b1000,service_b1000,tertiary_b1000,trunk_b1000,unclassified_b1000,motorway_b1500,primary_b1500,residential_b1500,secondary_b1500,service_b1500,tertiary_b1500,trunk_b1500,unclassified_b1500,motorway_b2000,primary_b2000,residential_b2000,secondary_b2000,service_b2000,tertiary_b2000,trunk_b2000,unclassified_b2000,motorway_b2500,primary_b2500,residential_b2500,secondary_b2500,service_b2500,tertiary_b2500,trunk_b2500,unclassified_b2500,motorway_b3000,primary_b3000,residential_b3000,secondary_b3000,service_b3000,tertiary_b3000,trunk_b3000,unclassified_b3000,motorway_b3500,primary_b3500,residential_b3500,secondary_b3500,service_b3500,tertiary_b3500,trunk_b3500,unclassified_b3500,motorway_b4000,primary_b4000,residential_b4000,secondary_b4000,service_b4000,tertiary_b4000,trunk_b4000,unclassified_b4000,motorway_b4500,primary_b4500,residential_b4500,secondary_b4500,service_b4500,tertiary_b4500,trunk_b4500,unclassified_b4500,motorway_b5000,primary_b5000,residential_b5000,secondary_b5000,service_b5000,tertiary_b5000,trunk_b5000,unclassified_b5000,contUrbFab_b25,discDensUrbFab_b25,discLowDensUrbFab_b25,discMedDensUrbFab_b25,discVLowDensUrbFab_b25,forest_b25,grUrbAreas_b25,herbVegetation_b25,industryCommercial_b25,otherRoads_b25,pastures_b25,railways_b25,sports_b25,annualCrops_b50,contUrbFab_b50,discDensUrbFab_b50,discLowDensUrbFab_b50,discMedDensUrbFab_b50,discVLowDensUrbFab_b50,forest_b50,grUrbAreas_b50,herbVegetation_b50,industryCommercial_b50,landNoUse_b50,otherRoads_b50,pastures_b50,railways_b50,sports_b50,water_b50,annualCrops_b100,contUrbFab_b100,discDensUrbFab_b100,discLowDensUrbFab_b100,discMedDensUrbFab_b100,discVLowDensUrbFab_b100,fastRoads_b100,forest_b100,grUrbAreas_b100,herbVegetation_b100,industryCommercial_b100,landNoUse_b100,mininDump_b100,otherRoads_b100,pastures_b100,railways_b100,sports_b100,water_b100,annualCrops_b300,constructionSites_b300,contUrbFab_b300,discDensUrbFab_b300,discLowDensUrbFab_b300,discMedDensUrbFab_b300,discVLowDensUrbFab_b300,fastRoads_b300,forest_b300,grUrbAreas_b300,herbVegetation_b300,industryCommercial_b300,isolStruct_b300,landNoUse_b300,mininDump_b300,otherRoads_b300,pastures_b300,railways_b300,sports_b300,water_b300,annualCrops_b500,constructionSites_b500,contUrbFab_b500,discDensUrbFab_b500,discLowDensUrbFab_b500,discMedDensUrbFab_b500,discVLowDensUrbFab_b500,fastRoads_b500,forest_b500,grUrbAreas_b500,herbVegetation_b500,industryCommercial_b500,isolStruct_b500,landNoUse_b500,mininDump_b500,otherRoads_b500,pastures_b500,permanentCrops_b500,railways_b500,sports_b500,water_b500,airports_b1000,annualCrops_b1000,constructionSites_b1000,contUrbFab_b1000,discDensUrbFab_b1000,discLowDensUrbFab_b1000,discMedDensUrbFab_b1000,discVLowDensUrbFab_b1000,fastRoads_b1000,forest_b1000,grUrbAreas_b1000,herbVegetation_b1000,industryCommercial_b1000,isolStruct_b1000,landNoUse_b1000,mininDump_b1000,opSpaceNoVeg_b1000,otherRoads_b1000,pastures_b1000,permanentCrops_b1000,railways_b1000,sports_b1000,water_b1000,airports_b1500,annualCrops_b1500,constructionSites_b1500,contUrbFab_b1500,discDensUrbFab_b1500,discLowDensUrbFab_b1500,discMedDensUrbFab_b1500,discVLowDensUrbFab_b1500,fastRoads_b1500,forest_b1500,grUrbAreas_b1500,herbVegetation_b1500,industryCommercial_b1500,isolStruct_b1500,landNoUse_b1500,mininDump_b1500,opSpaceNoVeg_b1500,otherRoads_b1500,pastures_b1500,permanen

In [28]:
df2.dtypes

CODI EOI                      int64
NOM ESTACIO                  object
FECHA                        object
O3                          float64
NO2                         float64
NO2_t_1                     float64
NO2_diff                    float64
TIPO_DIA                       bool
u10                         float64
v10                         float64
t2m                         float64
ssrd                        float64
tp                          float64
b25                         float64
b50                         float64
b100                        float64
b300                          int64
b500                          int64
b1000                         int64
b1500                         int64
b2000                         int64
b2500                         int64
b3000                         int64
b3500                         int64
b4000                         int64
b4500                         int64
b5000                         int64
residential_b25             

# Guardado DF

In [24]:
df2.to_csv('..\Datos\Datos_Procesados\DF_Mediciones_Predictores.csv')

In [25]:
filename = '..\Datos\Datos_Procesados\DF_Mediciones_Predictores'
outfile = open(filename,'wb')
pickle.dump(df2,outfile)
outfile.close()